<a href="https://colab.research.google.com/github/tMinnx/ProgrammingAssignment2_tMinnx/blob/master/8Aug2023Stochastic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Removes all objects from the current workspace
rm(list=ls())

## If deSolve package is not installed, you need to run the following line.
install.packages("deSolve")
install.packages("ggplot2")
library(deSolve)
library(ggplot2)


SIR compartments  
S is the number susceptible  
I is the number infectious  
R is the number recovered  
Total population N = S + I + R  

#[Part I Deterministic SIR model]
## Script A: for Question 1, 2 and 3
### Question 1
Calculate R0 for givenβ= 0.01 andγ=0.33.  
### Question 2
Let's run the R code to solve the deterministic SIR model.  
### Question 3
What condition can be required to determine epidemic sizes?  

In [ ]:
sir = function(time, y, params) {
    S=y[1]
    I=y[2]
    R=y[3]
    beta = params[["beta"]]
    gamma = params[["gamma"]]
    dS = -beta * S * I
    dI = beta * S * I - gamma * I
    dR = gamma * I

    solution = c(dS,dI,dR)
    return(list(solution))
}
## <<<Initial conditions>>>
## (S(0), I(0), R(0)) = (99, 1, 0)
S0 =99;I0=1;R0=0
N =S0+I0+R0
init = c(S =S0, I = I0, R=R0)
times = seq(0, 50, by = 0.1)
para = c(beta = 0.01, gamma = 0.33)

out = ode(y = init, times = times, func = sir, parms = para)
out = as.data.frame(out)
#Basic reproduction number: R0
(R0_repro=para[["beta"]]*S0/para[["gamma"]])
### Final size
# totalinf: The total number of infected cases
totalinf = tail(out$R, 1)
(final.size = totalinf/N)

### Figure 1
par(mfrow=c(1,2))
matplot(out$time, out[,2:4], type = "l", xlab = "Time", ylab = "Number of S(t), I(t), R(t)", main = "Deterministic SIR model", lwd = 2, lty = 1, bty = "l", col = 2:4)
legend(20,70, c("S(Susceptible)", "I(Infectious)", "R(Recovered)"), lwd=2, pch = 1, col = 2:4)
plot(out$time, out$I, xlab = "Time", ylab = "Number of I(t)",main = "Deterministic SIR model", lwd=1, pch = 1, col=3,xlim = c(0, 30), ylim = c(0,round(max(out$I))))


## Script A END
## Script for Question 1,2 and 3, select until here

#[Part II Stochastic SIR model]
## Script B: for Question 4, 5 and 6
### Question 4
What do you see in Plots Panel?  
Run the stochastic model selecting Script B.    
### Question 5
Does epidemics always take off if R0 > 1?  
You might see no epidemic.   
Why can't you reproduce the same figure?
### Question 6
Derive the probability of extinction and calculate 1/R0 to compare each.   

In [ ]:

## Stochastic SIR model using event-driven Gillespier's algorithm
gillespieSIR = function(beta, gamma, S0, I0, R0)
{
  times = S = I = R = c()
  # Step 1: initial condition:
  k        = 1
  S[k]     = S0
  I[k]     = I0
  R[k]     = R0
  times[k] = 0
  ## Run model:
  keepgoing = TRUE

  while(keepgoing)
  {
        # Step 2: calculate a1, a2
        infection = beta * S[k] * I[k]
        recovery = gamma * I[k]
        # Step 3-(1): random number of time until next reaction
        tau  = rexp(1, infection + recovery)
        # Step 3-(4): random number of the index of reaction
        eventtype  = sample(c('infection','recovery'), 1,
                            prob = c(infection,recovery))
        # Step 4: update time and state
        times[k+1] = times[k] + tau
        if(eventtype == 'infection')
        {
          S[k+1] = S[k] - 1
          I[k+1] = I[k] + 1
          R[k+1] = R[k]
        }
        if(eventtype == 'recovery')
        {
          S[k+1] = S[k]
          I[k+1] = I[k] - 1
          R[k+1] = R[k] + 1
        }
        k = k+1
        # Step 5: decide when to stop
        if(I[k] == 0) keepgoing = FALSE
  }
  output = data.frame(times = times, S = S, I = I, R = R)
  return(output)
}

### Figure 2
## (beta, gammma, S0, I0, R0) = (0.01, 0.33, 99, 1, 0)
model1 = gillespieSIR(0.01, 0.33, 99, 1, 0)
plot(model1$times, model1$I, xlab = "Time", ylab = "Number of I(t)", main = "Stochastic SIR Model", xlim = c(0, 30), ylim = c(0,60))


## Script C: for Question 7
### Question 7
Run Script C and see the figure on your Plots Panel.  
How does the figure on Plots Panel change, re-running Script C several times?  
Script C automatically simulates the model 20 times and output the figure containing all of these results.   


In [ ]:
num_sample <- 20
sto_sol<- list()
for (i in 1:num_sample) {
  ## input : (beta, gamma,S0, I0, R0)
  sto_sol[[i]] = gillespieSIR(0.01, 0.33, 99, 1, 0)
  sto_sol[[i]]$nbr = i
}

#Figure 3
Col <- rainbow(num_sample)
par(mfrow=c(1,1))
plot(sto_sol[[1]]$time, sto_sol[[1]]$I, type="l", col=Col[1],
     xlab = "Time", ylab = "Number of I(t)",
     main = "Stochastic SIR model",
     xlim = c(0, 30), ylim = c(0,60))
for (i in 2:num_sample) {
  lines(sto_sol[[i]]$time, sto_sol[[i]]$I, type = "l", col = Col[i])
}

#Figure 4
sto_allsol = do.call("rbind",sto_sol)
ggplot(sto_allsol,aes(x=times,y=I)) +
  geom_line(aes(x=times,y=I),linewidth=0.5) +
  scale_fill_brewer(palette="Set1")+
  coord_cartesian(xlim = c(0, 30), ylim = c(0,60))+
  #scale_y_continuous(expand=c(0,0),breaks=seq(0,2500,by=500))+
  facet_wrap(~nbr, scales="free")+
  theme_bw(base_size=15)+
  theme(panel.grid= element_blank())+
  labs(x="Time",y="Number of I(t)",title="Stochastic SIR model")


## Script D: for Question 8
###  Question 8
Select Script D and change the parameter β, and then run the program. Run the simulation 1000 times to calculate 50% quartile (median) for the total number of population infected with the stochastic SIR model as the following parameter β settings.    
Then, quartiles of the total number of infection will appear at the final line on your Console Panel.

In [ ]:
num_sample <- 20
sto_sol<- list()
for (i in 1:num_sample) {
  ## input : (beta, gamma,S0, I0, R0)
  sto_sol[[i]] = gillespieSIR(0.01, 0.33, 99, 1, 0)
}

sto_finalsize <- numeric()
for (i in 1:num_sample) {
  sto_finalsize[i] <- (tail(sto_sol[[i]],1)[["R"]])
}

#Figure 5
barplot(height = table(factor(sto_finalsize,0:100))*100/num_sample,col = "#539952",
        ylab = "Probability(%)",
        xlab = "Final size (total infected cases)",
        main = "Distribution of final size")

# Quartiles for the total population infected
quantile(sto_finalsize, p = c(0.025, 0.5, 0.975))

## Script E: for Question 8

In [ ]:
num_sample = 1000
S0=99;I0=1;R0=0;N=S0+I0+R0
betas = c(0.01,0.005,0.0033)
n_betas=length(betas)

sto_finalsize_all = data.frame(matrix(rep(0,num_sample*n_betas),ncol=n_betas))
for (k in 1:n_betas){
  for (i in 1:num_sample){
    sto_solution = gillespieSIR(betas[k], 0.33, S0,I0,R0)
    (sto_finalsize_all[i,k] <- tail(sto_solution, 1)[["R"]])
  }
}

freq_sto = list()
for (k in 1:n_betas){
   (freq_sto[[k]]=data.frame(beta=factor(betas[k]),table(factor(sto_finalsize_all[,k],0:N))/num_sample*100))
}
dist_final = do.call("rbind",freq_sto)
colnames(dist_final) = c("beta","cases","freq")

#Figure 6
  # barplot for distribution of final size
  fig_bar<-ggplot(dist_final,aes(x=cases,y=freq,fill=beta,color=beta)) +
    geom_bar(aes(x=cases,y=freq,fill=beta),stat="identity",position=position_dodge(),show.legend = TRUE) +
    scale_x_discrete(breaks=seq(0,N,by=10))+
    scale_fill_manual(name ="R0",labels = c("R0=3(beta=0.01)", "R0=1.5(beta=0.005)", "R0=1(beta=0.0033)"), values = c("red","blue","#539952")) +
    scale_color_discrete(guide=FALSE) +
    theme_bw(base_size=15)+
    theme(legend.position = c(0.8, 0.8),panel.grid= element_blank())+
    labs(x="Total infected cases",y="Probability (%)")
  print(fig_bar)

#Figure 7
  # points for distribution of final size
  fig_point<-ggplot(dist_final,aes(x=cases,y=freq,color=beta)) +
    geom_point(aes(x=cases,y=freq,color=beta,size=freq),show.legend = TRUE) +
    scale_x_discrete(breaks=seq(0,N,by=10))+
    scale_color_manual(name ="R0",labels = c("R0=3(beta=0.01)", "R0=1.5(beta=0.005)", "R0=1(beta=0.0033)"), values = c("red","blue","#539952")) +
    guides(colour = guide_legend(override.aes = list(size=5)))+
    theme_bw(base_size=12)+
    theme(legend.position = c(0.8, 0.5),panel.grid= element_blank())+
    labs(x="Total infected cases",y="Probability (%)")
  print(fig_point)


## Script F: for Question 9
### Question 9
Run Script F      
Make sure what the figures might imply compare to the first stochastic SIR model set-up as (β, γ, S(0), I(0), R(0)) = (0.01, 0.33, 99, 1, 0);   
1)	if initial outbreak size is bigger (initialized as S(0) = 79 and I(0) = 21)   

2)	if population size is bigger (initialized as S(0) = 999 and I(0) = 1)


In [ ]:
S0=79
I0=21
R0=0
##Stochastic model
num_sample <- 20
sto_sol<- list()
for (i in 1:num_sample) {
  ## input : (beta, gamma,S0, I0, R0)
  sto_sol[[i]] = gillespieSIR(0.01, 0.33, S0, I0, R0)
}
# Deterministic SIR model
  times <- seq(0, 30, by = 0.1)
  out <- as.data.frame(ode(y = c(S=S0, I=I0, R=R0), times = times, func = sir, parms = c(beta = 0.01, gamma = 0.33)))

#Figure 8
Col <- rainbow(num_sample)
plot(sto_sol[[1]]$times, sto_sol[[1]]$I, type = "l", col = Col[1],xlab = "Time", ylab = "Number of I(t)",main = paste0("(S(0),I(0))=(",S0,",",I0,")"),xlim = c(0, 30),ylim=c(0,round(max(out$I)*1.5)))
for (i in 2:num_sample) {
  lines(sto_sol[[i]]$times, sto_sol[[i]]$I, type="l", col=Col[i])
}
  # deterministic model
lines(times, out$I,type="l",col="red",lwd=2)